In [1]:
pip install datasets transformers rogue

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 17.5 MB/s eta 0:00:00
  Created wheel for rogue: filename=rogue-0.0.2-py3-none-any.whl size=7210 sha256=47461574d353c8e5d6e22359e071fb91255c28a953b972876c9fcbdc01d34882
  Stored in directory: /root/.cache/pip/wheels/11/db/ee/a44bd5d88bd4903f31c1f677a28ac79e354555bca48332f232
Successfully built rogue
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [2]:
pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=fc6eccdc2aa9b9892869ecd961156108a9bd0d0a4c424272d68a0b5d66625270
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [3]:
pip install rouge

BART MODEL

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from datasets import load_dataset
from rouge_score import rouge_scorer
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.translate.meteor_score import meteor_score

# Load the dataset
dataset = load_dataset("multi_news", split='test')

# Load the tokenizer and model
model_name = "facebook/bart-large-cnn"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
summarizer = pipeline("summarization", model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)

# Function to summarize and evaluate a single article
def summarize_and_evaluate(article, reference_summary):

    inputs = tokenizer(article, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = model.generate(inputs['input_ids'], max_length=250, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(reference_summary, generated_summary)

    return generated_summary, scores

rouge_results = []
for i, example in enumerate(dataset.take(5)):
    summary, scores = summarize_and_evaluate(example['document'], example['summary'])
    rouge_results.append({
        'id': i,
        'original_summary': example['summary'],
        'generated_summary': summary,
        'scores': scores
    })

# Display results
for result in rouge_results:
    print(f"Original Summary: {result['original_summary']}")
    print(f"Generated Summary: {result['generated_summary']}")
    print(f"ROUGE Scores: {result['scores']}\n")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

Original Summary: – It's a race for the governor's mansion in 11 states today, and the GOP could end the night at the helm of more than two-thirds of the 50 states. The GOP currently controls 29 of the country's top state offices; it's expected to keep the three Republican ones that are up for grabs (Utah, North Dakota, and Indiana), and wrest North Carolina from the Dems. That brings its toll to 30, with the potential to take three more, reports NPR. Races in Montana, New Hampshire, and Washington are still too close to call, and in all three, Democrat incumbents aren't seeking reelection. The results could have a big impact on health care, since a Supreme Court ruling grants states the ability to opt out of ObamaCare's Medicaid expansion. "A Romney victory would dramatically empower Republican governors," said one analyst. Click for NPR's state-by-state breakdown of what could happen.
Generated Summary: GOP Eyes Gains As Voters In 11 States Pick Governors. Eight of the gubernatorial 

In [ ]:
!pip install ipywidgets

Graphical user Interface for news summarization using BART model

In [6]:
# To display the generated summary it just takes a while. Please be patient.

import ipywidgets as widgets
from IPython.display import display

def summarize_article(b):
    # Get the input text from the text widget
    article_text = input_text.value

    # Call the summarize_and_evaluate function
    summary, _ = summarize_and_evaluate(article_text, "")

    # Set the generated summary to the output text widget
    output_text.value = summary

# Create input text widget
input_label = widgets.Label(value="Enter the article text:")
input_text = widgets.Textarea(layout={'height': '200px', 'width': '50%'})

# Create output text widget
output_label = widgets.Label(value="Generated Summary:")
output_text = widgets.Textarea(layout={'height': '100px', 'width': '50%'}, disabled=True)

# Create summarize button
summarize_button = widgets.Button(description="Summarize")
summarize_button.on_click(summarize_article)

# Display widgets
display(input_label, input_text, output_label, output_text, summarize_button)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.1 MB/s eta 0:00:00


Label(value='Enter the article text:')

Textarea(value='', layout=Layout(height='200px', width='50%'))

Label(value='Generated Summary:')

Textarea(value='', disabled=True, layout=Layout(height='100px', width='50%'))

Button(description='Summarize', style=ButtonStyle())

LSTM with Attention mechanism

In [4]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense, Attention
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from rouge import Rouge
from datasets import load_dataset

# Load the MultiNews dataset from Hugging Face
multi_news_dataset = load_dataset("multi_news")

df = pd.DataFrame(multi_news_dataset['test'])

# Rename columns for consistency
df.rename(columns={'document': 'document', 'summary': 'summary'}, inplace=True)

# Preprocess data
df['document'] = df['document'].apply(lambda x: x.lower())
df['summary'] = df['summary'].apply(lambda x: x.lower())

# Split dataset into train and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Tokenize text data
max_vocab_size = 10000
max_sequence_length = 100
tokenizer = Tokenizer(num_words=max_vocab_size)
tokenizer.fit_on_texts(train_df['document'])

train_sequences = tokenizer.texts_to_sequences(train_df['document'])
test_sequences = tokenizer.texts_to_sequences(test_df['document'])

train_padded = pad_sequences(train_sequences, maxlen=max_sequence_length, padding='post')
test_padded = pad_sequences(test_sequences, maxlen=max_sequence_length, padding='post')

vocab_size = min(max_vocab_size, len(tokenizer.word_index) + 1)

# Define model architecture
encoder_inputs = Input(shape=(max_sequence_length,))
encoder_embedding = Embedding(vocab_size, 128, input_length=max_sequence_length, mask_zero=True)(encoder_inputs)
encoder_lstm = LSTM(256, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(vocab_size, 128, input_length=max_sequence_length, mask_zero=True)(decoder_inputs)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)

attention = Attention()
context_vector = attention([decoder_outputs, encoder_outputs])
decoder_combined_context = tf.concat([context_vector, decoder_outputs], axis=-1)

decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_combined_context)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Train the model
early_stop = EarlyStopping(monitor='val_loss', patience=2, verbose=1)
model.fit([train_padded, train_padded[:, :-1]], train_padded[:, 1:], epochs=5, batch_size=64, validation_split=0.2, callbacks=[early_stop])



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/44972 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5622 [00:00<?, ? examples/s]

Epoch 1/5
57/57 [==============================] - 27s 307ms/step - loss: 7.4948 - val_loss: 7.0340
Epoch 2/5
57/57 [==============================] - 11s 185ms/step - loss: 6.9990 - val_loss: 6.9498
Epoch 3/5
57/57 [==============================] - 9s 163ms/step - loss: 6.8883 - val_loss: 6.8367
Epoch 4/5
57/57 [==============================] - 8s 129ms/step - loss: 6.7560 - val_loss: 6.7099
Epoch 5/5
57/57 [==============================] - 8s 139ms/step - loss: 6.6151 - val_loss: 6.5965


In [5]:
# Evaluate the model using ROUGE
from rouge_score import rouge_scorer

predictions = model.predict([test_padded, test_padded[:, :-1]])
predicted_summary = tokenizer.sequences_to_texts(np.argmax(predictions, axis=2))
actual_summary = test_df['summary']

predicted_summary = [summary.strip() for summary in predicted_summary]
actual_summary = [summary.strip() for summary in actual_summary]

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = {
    'rouge1': {'precision': [], 'recall': [], 'fmeasure': []},
    'rouge2': {'precision': [], 'recall': [], 'fmeasure': []},
    'rougeL': {'precision': [], 'recall': [], 'fmeasure': []}
}

# Calculate ROUGE scores for each summary pair
for pred, actual in zip(predicted_summary, actual_summary):
    score = scorer.score(actual, pred)
    for key in scores:
        scores[key]['precision'].append(score[key].precision)
        scores[key]['recall'].append(score[key].recall)
        scores[key]['fmeasure'].append(score[key].fmeasure)

avg_scores = {key: {metric: np.mean(values) for metric, values in metrics.items()} for key, metrics in scores.items()}

# Print average ROUGE scores for precision, recall, and fmeasure
print("Average ROUGE scores:")
for key, metrics in avg_scores.items():
    print(f"{key}:")
    for metric, score in metrics.items():
        print(f"  {metric}: {score:.4f}")

36/36 [==============================] - 5s 30ms/step
Average ROUGE scores:
rouge1:
  precision: 0.2962
  recall: 0.1320
  fmeasure: 0.1789
rouge2:
  precision: 0.0210
  recall: 0.0092
  fmeasure: 0.0125
rougeL:
  precision: 0.2633
  recall: 0.1187
  fmeasure: 0.1603


SENTENCE EMBEDDINGS

In [10]:
import datasets
import nltk
nltk.download('punkt')
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from rouge import Rouge
import tensorflow_hub as hub

# Load Multi-News dataset from Hugging Face
dataset = datasets.load_dataset("multi_news")

# Load Universal Sentence Encoder
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

rouge = Rouge()

def preprocess_text(text):
    sentences = nltk.sent_tokenize(text)
    return sentences
def generate_embeddings(document):
    sentences = preprocess_text(document)
    embeddings = embed(sentences)
    return embeddings
def compute_similarity_matrix(embeddings):
    similarity_matrix = cosine_similarity(embeddings)
    return similarity_matrix
def rank_sentences(similarity_matrix):
    scores = np.sum(similarity_matrix, axis=1)
    return scores.argsort()[::-1]

# Function to generate summary using sentence embeddings
def generate_summary(document, top_n=20):
    embeddings = generate_embeddings(document)
    similarity_matrix = compute_similarity_matrix(embeddings)
    ranked_sentences = rank_sentences(similarity_matrix)
    summary = " ".join([preprocess_text(document)[i] for i in ranked_sentences[:top_n]])
    return summary

# Function to evaluate summaries using ROUGE scores
def evaluate_rouge(reference, summary):
    scores = rouge.get_scores(summary, reference)
    return scores[0]

# Initialize an empty DataFrame to store ROUGE scores
df = pd.DataFrame(columns=[ 'ROUGE-1 Precision', 'ROUGE-1 Recall', 'ROUGE-1 F1',
                           'ROUGE-2 Precision', 'ROUGE-2 Recall', 'ROUGE-2 F1',
                           'ROUGE-L Precision', 'ROUGE-L Recall', 'ROUGE-L F1'])

#calculating averages for 10 instances because for the whole dataset it takes so long
count=0
for i, example in enumerate(dataset['test']):
    document = example["document"]
    reference_summary = example["summary"]
    generated_summary = generate_summary(document)

    rouge_scores = evaluate_rouge(reference_summary, generated_summary)

    # Add ROUGE scores to the DataFrame
    df.loc[i] = [
                 rouge_scores['rouge-1']['p'], rouge_scores['rouge-1']['r'], rouge_scores['rouge-1']['f'],
                 rouge_scores['rouge-2']['p'], rouge_scores['rouge-2']['r'], rouge_scores['rouge-2']['f'],
                 rouge_scores['rouge-l']['p'], rouge_scores['rouge-l']['r'], rouge_scores['rouge-l']['f']]

    count = count+1
    if count==10:
      break


avg_scores = df.mean()

# Print average ROUGE scores
print("Average ROUGE scores:")
print(avg_scores)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Average ROUGE scores:
ROUGE-1 Precision    0.203566
ROUGE-1 Recall       0.432881
ROUGE-1 F1           0.273025
ROUGE-2 Precision    0.058718
ROUGE-2 Recall       0.162113
ROUGE-2 F1           0.085179
ROUGE-L Precision    0.188574
ROUGE-L Recall       0.402363
ROUGE-L F1           0.253217
dtype: float64
